# Tutorial 8 (Enhanced): Scientific Features — Lineage + Reproducibility

We create a tracked session, perform operations, visualize the lineage graph, and reproduce results from stored lineage information.

In [ ]:
# Install
import sys, subprocess, pkgutil
for p in ['numpy','torch','matplotlib','seaborn','requests','networkx']:
    if pkgutil.find_loader(p) is None: subprocess.check_call([sys.executable,'-m','pip','install',p])
print('✅ Dependencies ready')

In [ ]:
# Setup and session
import time, json, hashlib, uuid, numpy as np, torch, networkx as nx
import matplotlib.pyplot as plt, seaborn as sns
from dataclasses import dataclass, field, asdict
from enum import Enum
from typing import Dict, Any, List
from datetime import datetime, timezone
sns.set_theme(style='whitegrid')
def checksum(s:str)->str: return hashlib.sha256(s.encode()).hexdigest()[:16]
class OpType(Enum): CRE='creation'; LA='linear_algebra'; DEC='decomposition'; AR='arithmetic'
@dataclass
class OpNode:
    node_id:str; op_type:OpType; name:str; params:Dict[str,Any]; inputs:List[str]; outputs:List[str]; timestamp:str; exec_ms:float; chk:str=''
    def __post_init__(self):
        if not self.chk: self.chk=checksum(self.name+str(self.params)+str(self.inputs))
@dataclass
class SciTensor: tid:str; data:torch.Tensor; creation_node:str; depth:int; meta:Dict[str,Any]=field(default_factory=dict)
class SciSession:
    def __init__(self): self.id=str(uuid.uuid4())[:8]; self.g=nx.DiGraph(); self.ops={}; self.tensors={}; print('🔬 Session:', self.id)
    def add(self, op:OpNode, data:torch.Tensor, meta=None): self.ops[op.node_id]=op; self.g.add_node(op.node_id, **asdict(op)); tid=f'tensor_{len(self.tensors):04d}'; self.tensors[tid]=SciTensor(tid,data,op.node_id,0,meta or {}); return self.tensors[tid]
    def edge(self, a:OpNode, b:OpNode): self.g.add_edge(a.node_id, b.node_id)
    def summary(self): return {'ops':len(self.ops),'tensors':len(self.tensors)}

## Pipeline and Lineage

In [ ]:
sess=SciSession()
op0=OpNode('create_0', OpType.CRE, 'create_tensor', {'shape':(32,32)}, [], ['t0'], datetime.now(timezone.utc).isoformat(), 0.5)
A=sess.add(op0, torch.randn(32,32), {'desc':'A'})
op1=OpNode('create_1', OpType.CRE, 'create_tensor', {'shape':(32,32)}, [], ['t1'], datetime.now(timezone.utc).isoformat(), 0.5)
B=sess.add(op1, torch.randn(32,32), {'desc':'B'}); sess.edge(op0, op1)
op2=OpNode('mm_2', OpType.LA, 'matmul', {}, ['t0','t1'], ['t2'], datetime.now(timezone.utc).isoformat(), 2.1)
C=sess.add(op2, A.data@B.data, {'desc':'C=A@B'}); sess.edge(op1, op2)
op3=OpNode('svd_3', OpType.DEC, 'svd_first_component', {}, ['t2'], ['t3'], datetime.now(timezone.utc).isoformat(), 3.5)
U=sess.add(op3, torch.linalg.svd(C.data).U, {'desc':'U(C)'})
op4=OpNode('add_4', OpType.AR, 'add', {}, ['t2','t2'], ['t4'], datetime.now(timezone.utc).isoformat(), 0.7)
D=sess.add(op4, C.data+C.data, {'desc':'D=C+C'}); sess.edge(op2, op3); sess.edge(op2, op4); sess.summary()

## Visualize Lineage Graph

In [ ]:
pos=nx.spring_layout(sess.g, seed=42)
plt.figure(figsize=(10,6)); nx.draw(sess.g, pos, with_labels=True, node_size=700, node_color='#8ecae6'); plt.title('Lineage'); plt.show()

## Reproduce from Lineage (Simulated)

In [ ]:
# Demonstrate how to rebuild D from A and B following lineage ops
A2=torch.randn(32,32); B2=torch.randn(32,32)
C2=A2@B2; U2=torch.linalg.svd(C2).U; D2=C2+C2
print('Shapes — C:',C2.shape,'U:',U2.shape,'D:',D2.shape)